Hi this is nice.

What now?

Wifi dataset: https://archive.ics.uci.edu/dataset/422/wireless+indoor+localization


In [ ]:
distance(v1, v2) = sqrt(sum((v1 - v2).^2))

In [ ]:
struct Point
    xn::Vector{Float16}
    label::String
end

function knn(X::Array{Point}, xn::Vector{Float16}, k::Int)
    ds = [distance(x.v, v) for x in X]
    return X[sortperm(ds)[1:k]]
end

In [ ]:
using CSV, DataFrames

wifi = CSV.read(
    "data/wifi.tsv",
    DataFrame,
    header=["w1", "w2", "w3", "w4", "w5", "w6", "w7", "room"]
)
wifi = wifi[(wifi.room .== 1) .| (wifi.room .== 2), :]

# Iterate across each row of wifi and create a Point for it
X = [
    Point(collect(row[[:w5, :w7]]), row[:room])
    for row in eachrow(wifi)
]

In [ ]:
using MLUtils

X_test, X_train = splitobs(X, at=0.15)
X_train = collect(X_train)
X_test = collect(X_test)

"Return the element that occurs most frequently in an array"
function majority(items::Vector{T})::T where T
    c = Dict{T, Int}()
    for it in items
        if !haskey(c, it)
            c[it] = 1
        else
            c[it] += 1
        end
    end
    return sort(collect(c), by=x->x[2], rev=true)[1][1]
end

# Compute the accuracy score
total = 0
correct = 0

for p in X_test
    neighbors = knn(X_train, p.v, 7)
    label = majority([x.label for x in neighbors])
    if label == p.label
        correct += 1
    end
    total += 1
end

println("Accuracy: $(correct / total * 100.0)%")